## To Do List - Have Comments:

### Importa Bibliotecas

In [1]:
import pandas as pd
import re
import numpy as np
from pathlib import Path
import glob
import warnings
from datetime import datetime

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.worksheet._reader")

### Carrega Dados Básicos

In [2]:
# Set the correct path to the Excel file
dados_basicos_filename = 'C:/Users/ELXY/Documents/Codigos/Python/P84_85/DadosBasicos/dados_basicos.xlsx'

# Read the Excel file
df_db = pd.read_excel(dados_basicos_filename, sheet_name=0)  # sheet_name=0 reads the first sheet
df_db = df_db.fillna('')
# display(df_db)

### Carrega todas as VDAs e todas as LDs
- Carrega as VDAs
- Carrega todas as LDs
- Associa aos campos de comentários das LDs comentadas 

In [3]:
vda_file = r'C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\VDA\\All_VDA.xlsx'
ld_file = r'C:\Users\ELXY\Documents\Codigos\Python\P84_85\LDs\LD_comment_combined.xlsx'

# Leitura dos arquivos 
df_vda = pd.read_excel(vda_file)
df_LD =  pd.read_excel(ld_file)

# Filtrar somente as últimas revisões de cada documento conforme coluna 'VDA VERSION'
def filtrar_ultimas_revisoes(df_vda):
    """
    Filtra o DataFrame para manter apenas as últimas revisões de cada documento.

    Parâmetros:
        df_vda (DataFrame): DataFrame contendo os dados do VDA.

    Retorna:
        DataFrame: DataFrame filtrado com as últimas revisões.
    """
    # Converte a coluna 'VDA VERSION' para string
    df_vda['Version'] = df_vda['Version'].astype(str)

    # Agrupa por 'Reference Document' e pega a última versão
    df_vda = df_vda.sort_values(by='Version').drop_duplicates(subset='Reference Document', keep='last')

    return df_vda
df_vda = filtrar_ultimas_revisoes(df_vda)

# Inclui no df_vda as seguintes informações do df_LD 'Discipline', 'Comments_new', 'PACKAGE' e 'Origem'. Os compos chaves de relacionamento são 'Reference Document' e 'CLIENT_DOCUMENT' dos dataframes df_vda e df_LD_final, respectivamente.
def incluir_informacoes(df_vda, df_LD):
    """
    Inclui informações do df_LD no df_vda com base na correspondência de 'Reference Document' e 'CLIENT_DOCUMENT'.
    
    Parâmetros:
        df_vda (DataFrame): DataFrame contendo os dados do VDA.
        df_LD (DataFrame): DataFrame contendo os dados dos comentários LD.
    
    Retorna:
        DataFrame: DataFrame atualizado com as novas colunas.
    """
    # Renomeia a coluna CLIENT_DOCUMENT para Reference Document para facilitar o merge
    df_LD = df_LD.rename(columns={'CLIENT_DOCUMENT': 'Reference Document'})
    
    # Realiza o merge
    df_vda = pd.merge(df_vda, df_LD[['Reference Document', 'Discipline', 'Comments_new', 'PACKAGE', 'Origem']],
                      on='Reference Document', how='left')
    
    return df_vda
df_vda = incluir_informacoes(df_vda, df_LD)
df_vda = df_vda.drop_duplicates(subset='Code', keep='last')

#### Criar um dataframe com somente os meus pacotes com base no df_db

In [4]:
# Com base no df_db filtrar somente os pacotes que estão no df_db coluna 'PKG_DESCRIPTION'
def filtrar_pacotes(df_vda, df_db):
    """
    Filtra o DataFrame df_vda para incluir apenas os pacotes que estão na coluna 'PKG_DESCRIPTION' do df_db.

    Parâmetros:
        df_vda (DataFrame): DataFrame contendo os dados do VDA.
        df_db (DataFrame): DataFrame contendo os pacotes a serem filtrados.

    Retorna:
        DataFrame: DataFrame filtrado com base nos pacotes do df_db.
    """
    pacotes = df_db['PKG_DESCRIPTION'].unique()
    return df_vda[df_vda['PACKAGE'].isin(pacotes)]
df_vda_meus_pacotes = filtrar_pacotes(df_vda, df_db)
# df_vda_meus_pacotes.info()


#### Carrega a ToDo List e associa ao dataframe df_vda_meus_pacotes

In [5]:
# Caminhos dos arquivos 
todo_list_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\To Do List.xlsx'
# Leitura dos arquivos 
df_todo = pd.read_excel(todo_list_file)

# Relacionar os dataframes df_todo com base no campo 'Related Object' do df_todo e 'Code' do df_vda_meus_pacotes
def relacionar_todo_vda(df_todo, df_vda):
    """
    Relaciona o DataFrame df_todo com df_vda_meus_pacotes com base no campo 'Related Object' do df_todo
    e 'Code' do df_vda_meus_pacotes.

    Parâmetros:
        df_todo (DataFrame): DataFrame contendo a lista de tarefas.
        df_vda (DataFrame): DataFrame contendo os dados do VDA filtrados.

    Retorna:
        DataFrame: DataFrame atualizado com as novas colunas.
    """
    # Renomeia a coluna 'Related Object' para 'Code' para facilitar o merge
    df_todo = df_todo.rename(columns={'Related Object': 'Code'})

    # Realiza o merge
    df_todo = pd.merge(df_todo, df_vda[['Code', 'Reference Document', 'Title', 'Discipline', 'Comments_new', 'PACKAGE', 'Origem']],
                       on='Code', how='left')

    return df_todo
df_todo = relacionar_todo_vda(df_todo, df_vda_meus_pacotes)


#### Cria a coluna 'Additional Info' para colar no ToDo list do Integra

In [6]:
# Criar ou atualizar a coluna 'Additional Info' com as informações das colunas 'Code', 'Reference Document', 'Title', 'Discipline', 'Comments_new' do df_todo. Com a seguinte lógica:
# Se a coluna 'Additional Info' já existir, ela deve ser atualizada com as novas informações. Se não existir, ela deve ser criada.
# Se o valor da coluna 'Comments_new' for igual a 'YES', a coluna 'Additional Info' deve ser preenchida com as seguinte formatação como exemplo: "VDA: VDA-P84-AP5-00049 DOCUMENT: I-MC-3010.2S-5121-940-D5A-006 UE-5121501 - SEA WATER ELECTROCHLORINATION UNIT - HYDRAULIC CALCULATIONS, Comment: YES, Discipline: Process  , Date: 25-June-2025".
# Se o valor da coluna 'Comments_new' não tiver informação ou NaN, a coluna 'Additional Info' deve ser preenchida com as seguinte formatação como exemplo: "VDA: VDA-P84-AP5-00049 DOCUMENT: I-DE-3010.2S-1231-296-AP5-065 NOZZLES AND SADDLE DETAIL DRAWING FOR GAS PIPELINE PIG LAUNCHER / RECEIVER (LR-1231001), Comment: NO, Date: 25-June-2025".
# Se o valor da coluna 'Task Name' != 'VDA have a Comments', a coluna 'Additional Info' deve ser preenchida com as seguinte formatação como exemplo: 'NÃO É HAVE A COMMENT'
def atualizar_additional_info(df_todo):
    """
    Atualiza ou cria a coluna 'Additional Info' no DataFrame df_todo com base nas regras especificadas.

    Parâmetros:
        df_todo (DataFrame): DataFrame contendo a lista de tarefas.

    Retorna:
        DataFrame: DataFrame atualizado com a coluna 'Additional Info'.
    """
    def format_additional_info(row):
        if row['Comments_new'] == 'YES':
            return f"{row['Code']} DOCUMENT: {row['Reference Document']} - {row['Title']}, Comment: YES, Discipline: {row['Discipline']}, Date: {datetime.now().strftime('%d-%B-%Y')}"
        elif pd.isna(row['Comments_new']) or row['Comments_new'] == '':
            return f"{row['Code']} DOCUMENT: {row['Reference Document']} - {row['Title']}, Comment: NO, Date: {datetime.now().strftime('%d-%B-%Y')}"
        elif row['Task Name'] != 'VDA have a Comments':
            return 'NÃO É HAVE A COMMENT'
        else:
            return ''

    df_todo['Additional Info'] = df_todo.apply(format_additional_info, axis=1)
    return df_todo
df_todo = atualizar_additional_info(df_todo)

# Salvar o arquivo atualizado
todo_list_output = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\To Do List_Comments.xlsx'
df_todo.to_excel(todo_list_output, index=False, engine='openpyxl')

print(f'Arquivo To Do List com comentários salvo em: {todo_list_output}')

Arquivo To Do List com comentários salvo em: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\To Do List_Comments.xlsx
